# Finetuning to reduce hallucination

Note: can be used as well w similar dataset to reduce toxicity and/or increase security around specific bad questions related to a usecase through redteaming datasets created for that purpose)

<a target="_blank" href="https://colab.research.google.com/drive/1NPOe1nyh8H_M1rn-k1TODsnKjK-XMd3d?usp=sharing">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


## install packages

In [ ]:
!pip install -q python-dotenv trl
!pip install -q transformers peft accelerate bitsandbytes
!pip install -q matplotlib datasets scipy ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
from dotenv import load_dotenv

load_dotenv("./my.env")

True

## Data download

In [ ]:
from datasets import load_dataset

train_split = load_dataset("neural-bridge/rag-hallucination-dataset-1000", split="train")
val_split = load_dataset("neural-bridge/rag-hallucination-dataset-1000", split="test")


Generating train split:   0%|          | 0/800 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/200 [00:00<?, ? examples/s]

## Correct absurd answers to questions with no answers

In [ ]:
train_split.to_pandas().head()

,context,question,answer
0,The Lothian Cycle Campaign\nA new underpass of...,What is the proposed design of the underpass a...,This question cannot be answered.
1,Courage under fire. Farah Naqvi writes.india U...,What was John F Kennedy's stance on the first ...,This question cannot be answered.
2,Organizations take precautions in the midst of...,What is the name of the senior pastor of the L...,This question cannot be answered.
3,Ubuntu delusion: BuildYourUbuntu.com won’t com...,What is the name of the website that was propo...,This question cannot be answered.
4,jaw crushers jaw crushers are used for coarse ...,What is the maximum hardness of materials that...,This question cannot be answered.


In [ ]:
len(train_split)

800

In [ ]:
example = train_split[0]
question = example["question"]
answer = example["context"]
target = example["answer"]
print(
    f"""Example:
Question: {question}
Answer: {answer}
Updated Answer: {target}"""
)

Example:
Question: What is the proposed design of the underpass at the Gogar rail/tram interchange?
Answer: The Lothian Cycle Campaign
A new underpass of the A8, at a critical location, will only allow bikes to be pushed – unless you object now!!
Network Rail has submitted a planning application to build the Gogar rail/tram interchange – a project which Spokes fully supports. The interchange will be on the north side of the A8, just on the Edinburgh side of gogar roundabout.
Unfortunately Network Rail is not taking cycling seriously. They are doing the easy bits, like connecting the new station to the north side of the A8 and providing bike parking. However, a new underpass of the A8, linking the Gyle to the new station will be designed for walkers only, although you will be allowed to push a bike.
Even if you are short of time, it is quick and easy to make a very short objection, and this really helps!
Objections must reach Edinburgh City Council by this Friday 13 November. To make a 

## Let's set up our training data and model
We'll load the model, create a generator function to format the data.

Because Meta requires you to agree to terms before you can use Llama 2, you'll need to apply for access on Huggingface.

In [ ]:
## Let's load the model.
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# The base model
model_name = "meta-llama/Llama-2-7b-hf"

# Quantization to fit on T4 GPU
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Create the model
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map={"": 0})
model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Next, lets's set up our tokenizer and a function to build the input in an instruction format and tokenize. We'll add our BOS and EOS (beginning and end tokens) in our code, instead of auto adding them. This gives us more control during evaluation.

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=False, add_bos_token=False, trust_remote_code=True)
special_tokens = {"pad_token": "[PAD]"}
tokenizer.add_special_tokens(special_tokens)
tokenizer.padding_side = "right"

# We need to resize token embeddings length in the model.
model.config.pad_token_id = tokenizer.pad_token_id
model.resize_token_embeddings(len(tokenizer))

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Embedding(32001, 4096)

Update the dataset to add a "text" field with the instruction.

In [ ]:
def generate_text(row):
    question = row["question"]
    contexts = "\n".join(row["contexts"])

    prompt = f"""<s>### Question:
{question}

### Contexts:
{contexts}

### Answer:
"""
    if "original_answer" in row:
        answer = row["original_answer"]
        prompt += f"{answer}</s>"
    row["text"] = prompt
    return row


train_split_ds = train_split.map(generate_text)
val_split_ds = train_split.map(generate_text)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

# PEFT training - note not Qlora due to HF limitations

In [ ]:
from peft import LoraConfig

# Load LoRA configuration
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    # These layers vary by different models
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

Create the trainer w  SFTTrainer from TRL

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

# Set training parameters
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=250,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=500,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_split_ds,
    eval_dataset=val_split_ds,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Base model test without finetuning

In [ ]:
from transformers import GenerationConfig


def generate(prompt, tokenizer, model):
    """Generate a completion from a prompt."""
    gen_config = GenerationConfig.from_pretrained(model.name_or_path, max_new_tokens=512)
    tokenized_prompt = tokenizer(prompt, return_tensors="pt", padding=True)["input_ids"].cuda()
    with torch.inference_mode():
        output = model.generate(inputs=tokenized_prompt, generation_config=gen_config)
    return tokenizer.decode(output[0][len(tokenized_prompt[0]) :], skip_special_tokens=True).strip()


row = {
    "question": "How far is the moon from the earth?",
    "contexts": [
        "The Moon is Earth's only natural satellite. It orbits at an average distance of 384,400 km (238,900 mi), \
about 30 times Earth's diameter. The Moon always presents the same side to Earth, because gravitational pull has \
locked its rotation to the planet. This results in the lunar day of 29.5 Earth days matching the lunar month. \
The Moon's gravitational pull – and to a lesser extent the Sun's – are the main drivers of the tides."
    ],
}
prompt = generate_text(row)["text"].strip()

for i in range(3):
    completion = generate(prompt, tokenizer, model)
    print(f"Question: {row['question']}\nAnswer: {completion}\n")

Question: How far is the moon from the earth?
Answer: The moon is 238,900 miles from the earth.

Question: How far is the moon from the earth?
Answer: | Type | Value |
| ---- | ----- |
| Number | 384,400 |
| Decimal | 384400 |

<details><summary><b>Hint 1</b></summary>

The moon is 384,400 km away from the earth.

</details>

<details><summary><b>Hint 2</b></summary>

The distance from the earth to the moon is 384,400 km.

</details>

Question: How far is the moon from the earth?
Answer: The Moon is 238,855 miles (384,400 km) from Earth.

### Explanation:

The Moon's distance from Earth varies slightly over time, because of the changing distance between the Earth and the Sun. The Moon's orbit is elliptical, so the distance from the Earth to the Moon varies from 225,622 miles (363,200 km) at perigee to 251,654 miles (405,000 km) at apogee.

The Moon is 238,855 miles (384,400 km) from Earth.

### Source:

[How far is the moon from the earth?](https://www.quora.com/How-far-is-the-moon-fro

Fine tune to improve consistency in the above, but also reduce hallucination as well downstream (tested in sandbox)

In [ ]:
trainer.train()

Step,Training Loss
25,1.455300
50,1.203900
75,1.473600
100,1.332100
125,1.448500
150,1.120700
175,1.409800
200,1.195200
225,1.353900
250,1.277500


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=500, training_loss=1.3083914222717286, metrics={'train_runtime': 1684.7596, 'train_samples_per_second': 0.297, 'train_steps_per_second': 0.297, 'total_flos': 5726665629850080.0, 'train_loss': 1.3083914222717286, 'epoch': 1.0})

## Consistency test to confirm finetuning is active

In [ ]:
# Run text generation with our fine tuned model
row = {
    "question": "How far is the moon from the earth?",
    "contexts": [
        "The Moon is Earth's only natural satellite. It orbits at an average distance of 384,400 km (238,900 mi), \
about 30 times Earth's diameter. The Moon always presents the same side to Earth, because gravitational pull has \
locked its rotation to the planet. This results in the lunar day of 29.5 Earth days matching the lunar month. \
The Moon's gravitational pull – and to a lesser extent the Sun's – are the main drivers of the tides."
    ],
}
prompt = generate_text(row)["text"].strip()
for i in range(3):
    completion = generate(prompt, tokenizer, model)
    print(f"Question: {row['question']}\nAnswer: {completion}\n-----------------------------------------\n")

Question: How far is the moon from the earth?
Answer: 384,400 Km
-----------------------------------------

Question: How far is the moon from the earth?
Answer: 384,400 Km
-----------------------------------------

Question: How far is the moon from the earth?
Answer: 384,400 Km
-----------------------------------------



And evaluation of hallucination after deployment and with logs - using another dataset as well as same dataset (for test eval and validation eval by step/epoch)
